### Задание 2.
Имеется таблица, содержащая лог размещения и перемещения банкоматов по локациям.
Один банкомат может быть размещен по различным адресам в разные моменты времени.

таблица: ATM\
ATM_ID  iNTEGER     Идентификатор\
ADR     STRING      Адрес размещения\
DT_INS  TIMESTAMP        Дата и время установки по указанному адресу

Сформировать отчет по состоянию на 01.01.2020 00:00:00 с перечнем банкоматов: их идентификатор, адрес размещения и датой установки

ATM_ID;ADR;DT_INS\
1;Ленина,2;2019.01.01 00:01:00\
1;Ленина,3;2019.06.01 00:01:00\
1;Ленина,2;2019.12.21 00:01:00\
1;Ленина,5;2020.12.21 00:01:00\
2;Ленина,4;2019.12.01 00:01:00\
2;Ленина,4;2019.12.01 00:01:00\
10;Ленина,5;2019.08.01 00:01:00\
15;Ленина,6;2019.07.01 00:01:00\

В данных могут быть проблемы - минусом отмечены некорректные строки лога (подразумевается, что нужно понять логику некоректности). \
Плюсом отмечена корректная строка по данному банкомату (без маркеров - также корректные строки).
Для построения отчета необходимо брать только корректные.

1;Ленина,2;2019.01.01 00:01:00\
1;Ленина,3;2019.06.01 00:01:00\
1;Ленина,2;2019.12.21 00:01:00\
1;Ленина,5;2020.12.21 00:01:00\
+2;Ленина,2;2019.01.01 00:01:00\
-2;Ленина,2;2019.06.01 00:01:00\
-2;Ленина,2;2019.12.21 00:01:00\
2;Ленина,3;2020.12.21 00:01:00

Задание выполняется на SQL и на python.\
Для проверки задания также необходимо подготовить скрипт на python для генерации тестовых данных объемом не менее 1млн. строк. Названия улиц не обязательно должны быть осмысленными, главное чтобы выполнялось условие про некорректные строки, а решение должно предусматривать выполнение обозначенной логики построения отчета.\  
С учетом того, что скрипт генерирует каждый раз новые данные, достаточно чтобы в среднем (при нескольких повторениях) скрипт генерировал в т.ч. некорректные строки.

In [1]:
import pandas as pd

Данные сгенерированы в отдельном скрипте и сохранены в csv. Загрузим их

In [12]:
task2_data = pd.read_csv('~/!Tests/task2_data.csv', sep=';', parse_dates=['DT_INS']).drop(columns=['Unnamed: 0'])
task2_data

,ATM_ID,ADR,DT_INS
0,1,"Заречная, 16",2019-10-19 21:37:00
1,1,"Заречная, 20",2019-11-22 20:53:48
2,1,"Заречная, 21",2019-11-02 22:05:48
3,1,"Зеленая, 16",2019-10-27 06:58:36
4,1,"Зеленая, 26",2019-11-26 22:05:48
...,...,...,...
999995,39910,"Новая, 14",2019-11-18 14:10:36
999996,39916,"Садовая, 28",2019-09-12 19:13:00
999997,39917,"Садовая, 27",2019-11-11 12:15:24
999998,39919,"Мятная, 23",2019-11-01 21:08:12


Сформируем отчет.

In [23]:
#оставим только строки до отчетной даты
report = task2_data.query('DT_INS < @pd.to_datetime("2020-01-01")')
#подтянем адрес предыдущего положения банкомата по времени по каждому id банкомата отдельно
report['ADR_prev'] = report.sort_values('DT_INS').groupby('ATM_ID')['ADR'].shift(1)
#исключим строки, текущий и предыдущий адрес по которым совпадает
report = report.query('ADR != ADR_prev or ADR_prev != ADR_prev')
#отранжируем по времени в обратном порядке и удалим дубликаты по ATM_ID
report = report.sort_values('DT_INS', ascending=0).drop_duplicates(subset='ATM_ID')
#получим итоговый отчет
report.drop(columns=['ADR_prev']).sort_values('ATM_ID').head(10)

C:\Games\Anaconda\Anaconda installed\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,ATM_ID,ADR,DT_INS
7,1,"Лазо, 20",2019-12-26 12:58:36
48,2,"Заречная, 13",2019-12-31 02:39:24
152,3,"Зеленая, 1",2019-12-28 18:58:36
312,4,"Ромашковая, 10",2019-12-31 04:20:12
362,5,"Ленина, 28",2019-12-29 07:13:00
452,6,"Лазо, 21",2019-12-29 13:41:48
616,7,"Ромашковая, 22",2019-12-29 01:56:12
664,8,"Лазо, 14",2019-12-31 04:34:36
791,9,"Победы, 15",2019-12-31 12:29:48
824,10,"Заречная, 20",2019-12-31 12:44:12


Проверка:

In [22]:
task2_data.query('DT_INS < @pd.to_datetime("2020-01-01")')['ATM_ID'].nunique()

38206

In [24]:
report['ATM_ID'].nunique()

38206

Все ок.